In [1]:
import csv
import pandas as pd
from collections import defaultdict, Counter
import re

- row with wrong number of fields ++
- blank lines ++
- new line characters ++
- unclosed quotes ++
- invalid incoding (odd characters that cause encoding errors) ++
- duplicated column names ++
- duplicated IDs ++

In [2]:
# Check encoding
encoding_list = ['utf-8','latin-1','cp1252','utf32']

for encode in encoding_list:
    try:
        f = open('example.csv','r', encoding=encode)
        data = f.readlines()
        f.close()
        break
    except UnicodeDecodeError:
        print('Cannot decode with', encode)

if data:
    print('Using encoding', encode)
else:
    print('None of the encodings worked')
    

Using encoding utf-8


In [3]:
# May add other forms of state abbreviations (i.e. California, ca)
states = set(["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
              "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
              "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
              "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
              "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"])

In [4]:
def new_search_list(row, headers):
    # initialize
    new_row = ['']*(len(headers)+1)
    
    # search for state, zip-code, website. These are features with patterns
    state = list(set(row[1:]).intersection(states))
    zipcode = [x for x in row[1:] if re.match('^\d{5}(?:[-]\d{4})?$',x)!=None]
    website = [w for w in row[1:] if 'www.' in w]
    
    search_list = [state, zipcode, website]
    search_new = []
    
    for search in search_list:
        if search:
            search_new.append(search[0])
        else:
            search_new.append('')
    
    # redefine row ()
    new_row[:3] = row
    new_row = new_row[:3] + search_new
    return new_row

In [5]:
reader = csv.reader(data, delimiter=",", dialect=csv.excel_tab)
id_dict = {}
dup_id = defaultdict(int)
diff_len_rows = []

# Check for duplicate column names
headers = next(reader, None)
if headers:
    header_counts = Counter(headers)  
    for s, num in header_counts.items():
        if num>1:
            for suffix in range(1,num+1):
                headers[headers.index(s)] = s + str(suffix)
headers = headers[1:] 

for row in reader:
    # Check new lines and unclosed quotes
    row = [x.replace('"s','\'s').replace('"','').replace('\n',' ') for x in row]
    
    # Skip empty rows
    if len(row)>0:
        
        # Handling rows with wrong number of fields (greater or less than columns)
        if len(row) != len(headers)+1:
            # We still might want to check manually
            diff_len_rows.append(row)
            # Attempt to fix it
            row = new_search_list(row, headers)
           
        # Check duplicate IDs
        if len(row)==len(headers)+1:
            # if ID doesn't exist, or ID exists but values are different
            if (row[0] not in id_dict) or ((row[0] in id_dict) and id_dict[row[0]] == {key: value for key, value in zip(headers, row[1:])}):
                # add to dictionary
                id_dict[row[0]] = {key: value for key, value in zip(headers, row[1:])}
            else:
                # add to dictionary with encounter suffix (i.e. 34_1, 34_2)
                dup_id[row[0]]+=1
                id_dict[row[0] + '_' + str(dup_id[row[0]])] = {key: value for key, value in zip(headers, row[1:])}   
